In [ ]:
pip install --upgrade pip

In [ ]:
%%capture
!pip install bertopic

In [ ]:
pip install farm-haystack[colab,faiss]

In [ ]:
pip install bertopic

In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import re
import pickle
import numpy as np

In [ ]:
from bertopic import BERTopic

In [ ]:
import pprint

In [ ]:
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
import re
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data = pd.read_csv('bbc-news-data.csv', sep='\t')

In [ ]:
data

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


In [ ]:
def clean_text(sentence):
        # Remove Non Alphanumeric sequences

        pattern = re.compile(r'[^a-z]+')
        sentence = sentence.lower()
        sentence = pattern.sub(' ', sentence).strip()

        # Tokenize

        word_list = word_tokenize(sentence)

        # Stop Words & Punctuation

        stopwords_list = set(stopwords.words('english'))
        punct = set(punctuation)

        # Remove stop words, very small words & punctuation
        word_list = [word for word in word_list if word not in stopwords_list]
        word_list = [word for word in word_list if len(word) > 2]
        word_list = [word for word in word_list if word not in punct]

        # Stemming or Lemmatization
        lemmer = WordNetLemmatizer()

        word_list = [lemmer.lemmatize(word) for word in word_list]

        sentence = ' '.join(word_list)

        return sentence

def create_dataframe(save=False):
  tqdm.pandas()
  temp_dataset = pd.read_csv('./bbc-news-data.csv', sep='\t')
  documents_df = pd.DataFrame({
      'file': temp_dataset['filename'],
      'title': temp_dataset['title'],
      'target': temp_dataset['category'],
      'docs': temp_dataset['content'],
    }
  )

  documents_df['docs_clean'] = documents_df['docs'].progress_apply(lambda x: clean_text(str(x)))

  documents_df = documents_df[documents_df['docs_clean'].notna()]
  return documents_df

In [ ]:
documents = create_dataframe()

100%|██████████| 2225/2225 [00:08<00:00, 265.21it/s]


In [ ]:
documents

,file,title,target,docs,docs_clean
0,001.txt,Ad sales boost Time Warner profit,business,Quarterly profits at US media giant TimeWarne...,quarterly profit medium giant timewarner jumpe...
1,002.txt,Dollar gains on Greenspan speech,business,The dollar has hit its highest level against ...,dollar hit highest level euro almost three mon...
2,003.txt,Yukos unit buyer faces loan claim,business,The owners of embattled Russian oil giant Yuk...,owner embattled russian oil giant yukos ask bu...
3,004.txt,High fuel prices hit BA's profits,business,British Airways has blamed high fuel prices f...,british airway blamed high fuel price drop pro...
4,005.txt,Pernod takeover talk lifts Domecq,business,Shares in UK drinks and food firm Allied Dome...,share drink food firm allied domecq risen spec...
...,...,...,...,...,...
2220,397.txt,BT program to beat dialler scams,tech,BT is introducing two initiatives to help bea...,introducing two initiative help beat rogue dia...
2221,398.txt,Spam e-mails tempt net shoppers,tech,Computer users across the world continue to i...,computer user across world continue ignore sec...
2222,399.txt,Be careful how you code,tech,A new European directive could put software w...,new european directive could put software writ...
2223,400.txt,US cyber security chief resigns,tech,The man making sure US computer networks are ...,man making sure computer network safe secure r...


BERTopic Topic Model

In [ ]:
topic_model = BERTopic(
    embedding_model='sentence-transformers/gtr-t5-base',
    top_n_words = 15,
    n_gram_range = (1,4)
)

NameError: ignored

In [ ]:
topics, probas = topic_model.fit_transform(documents['docs_clean'])

In [ ]:
topic_model.visualize_topics()

In [ ]:
all_topics = topic_model.get_topics()

bertopic_keywords = []
for topic, keys in all_topics.items():
  keywords = []
  for k in keys: keywords.append(k[0])

  bertopic_keywords.append(keywords)

  print(f'Topic: {topic}, {keywords}')

Topic: -1, ['said', 'mobile', 'technology', 'service', 'people', 'phone', 'digital', 'gadget', 'network', 'computer', 'net', 'one', 'new', 'also', 'make']
Topic: 0, ['said', 'would', 'government', 'year', 'labour', 'party', 'also', 'election', 'minister', 'new', 'people', 'say', 'could', 'company', 'country']
Topic: 1, ['club', 'chelsea', 'game', 'arsenal', 'player', 'liverpool', 'goal', 'league', 'said', 'united', 'manager', 'football', 'time', 'team', 'would']
Topic: 2, ['film', 'best', 'award', 'actor', 'oscar', 'director', 'star', 'year', 'actress', 'movie', 'said', 'festival', 'role', 'also', 'aviator']
Topic: 3, ['england', 'wale', 'ireland', 'rugby', 'game', 'six nation', 'side', 'six', 'nation', 'france', 'half', 'player', 'coach', 'try', 'win']
Topic: 4, ['music', 'band', 'album', 'song', 'year', 'chart', 'best', 'single', 'one', 'award', 'said', 'number', 'singer', 'artist', 'rock']
Topic: 5, ['open', 'roddick', 'set', 'match', 'final', 'seed', 'win', 'australian', 'year', 'f

In [ ]:
topics[:50]

Zero Shot Classification of BERTopic Topics

In [ ]:
from transformers import pipeline

In [ ]:
topic_classifier_pipeline = pipeline ("zero-shot-classification", model = "facebook/bart-large-mnli")

In [ ]:
topic_labels = ['business', 'entertainment', 'politics', 'sport', 'tech']

In [ ]:
bertopics = []
for topic in bertopic_keywords:
  bertopics.append(' '.join(topic))

In [ ]:
input_sequence = bertopics[0]
# label_candidate = ['travel', 'cooking', 'entertainment', 'dancing', 'technology']

In [ ]:
res = []

for topic in bertopics:
  res.append(topic_classifier_pipeline(topic, topic_labels, multi_label=True))

In [ ]:
res

[{'sequence': 'said mobile technology service people phone digital gadget network computer net one new also make',
  'labels': ['tech', 'entertainment', 'business', 'sport', 'politics'],
  'scores': [0.9933239221572876,
   0.5606679320335388,
   0.4206012189388275,
   0.0004904124070890248,
   6.929949449840933e-05]},
 {'sequence': 'said would government year labour party also election minister new people say could company country',
  'labels': ['politics', 'business', 'entertainment', 'tech', 'sport'],
  'scores': [0.9950567483901978,
   0.19063186645507812,
   0.05536574125289917,
   0.024503488093614578,
   0.00040458087460137904]},
 {'sequence': 'club chelsea game arsenal player liverpool goal league said united manager football time team would',
  'labels': ['sport', 'business', 'entertainment', 'tech', 'politics'],
  'scores': [0.9950889348983765,
   0.8514499664306641,
   0.676670491695404,
   0.4558907747268677,
   0.00015637830074410886]},
 {'sequence': 'film best award actor 

In [ ]:
mutli = []
for i,topic in enumerate(res):
  if topic['scores'][1] > 0.8:
    print(f'Topic {i-1}: Multi')
    mutli.append(True)
  else:
    mutli.append(False)

Topic 1: Multi
Topic 6: Multi
Topic 10: Multi
Topic 13: Multi
Topic 16: Multi
Topic 20: Multi
Topic 21: Multi


BERTopic Accuracy Evaluation

In [ ]:
pos = 0

for doc, doc_topic in enumerate(topics):
  if mutli[doc_topic+1] == True:
    if documents.iloc[doc].target in res[doc_topic+1]['labels'][:2]:
      pos += 1
  else:
    if documents.iloc[doc].target == res[doc_topic+1]['labels'][0]:
      pos += 1

In [ ]:
accu = pos / len(topics)

print(f'Zero Shot Topic Classification Accuracy: {round(accu*100,2)}%')

Zero Shot Topic Classification Accuracy: 71.37%


QnA Subsystem

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import EmbeddingRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers
from haystack.nodes import TransformersQueryClassifier

In [ ]:
business_document_store = FAISSDocumentStore(faiss_index_factory_str='Flat')

In [ ]:
business_docs = documents.loc[documents.target == 'business']

In [ ]:
for topic in topic_labels:
  globals()[f'{topic}_documents'] = documents.loc[documents.target == topic]
  globals()[f'{topic}_document_store'] = FAISSDocumentStore(faiss_index_factory_str='Flat')

  data = documents.loc[documents.target == topic]

  for index, row in data.iterrows():
    globals()[f'{topic}_document_store'].write_documents([{
        "content": row["docs"],
        "meta": {
            "tags": row["target"],
            'title': row['title']
        }
    }])

  # globals()[f'{topic}_document_store'].write_documents(data)
  globals()[f'{topic}_docuemnt_store'].update_embeddings()



Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

KeyError: ignored

In [ ]:
# for topic in topic_list:
#   globals()['docs_topic_%s' % topic] = docs_df.docs_clean.loc[docs_df["projected_topic"] == topic]
#   globals()['embs_topic_%s' % topic] = emb_df.iloc[docs_df.index[docs_df["projected_topic"] == topic]].to_numpy()

#   globals()['subtopic_model_%s' % topic] = BERTopic()
#   globals()['subtopic_model_%s_topics' % topic], globals()['subtopic_model_%s_probas' % topic] = globals()['subtopic_model_%s' % topic].fit_transform(globals()['docs_topic_%s' % topic], globals()['embs_topic_%s' % topic])

# subtopic_model_computers = BERTopic().fit(docs_topic_computers, embs_topic_computers)

In [ ]:
# for index, row in documents.iterrows():
#     document_store.write_documents([{
#         "content": row["docs"],
#         "meta": {
#             "tags": row["target"],
#             'title': row['title']
#         }
#     }])

In [ ]:
documents